# Day 10

This notebook is designed to cover some of the requests from [this](https://piazza.com/class/iju1ffuooio7mp?cid=28) Piazza post.

High-level contents of this notebook:
1.  Visualizing models
2.  Determining when to use a particular model
3.  Interpreting log-loss error metric

## Loading the Data

We will be using both the Rotten Tomatoes and the SF Crime datasets as running examples.  This notebook assumes that you have downloaded the data for these contests from Kaggle and stored them in the directories indicated below.

In [ ]:
import pandas as pd

tomatoes = pd.read_csv('../../datasets/rotten_tomatoes_train.tsv', sep='\t')
crime = pd.read_csv('../../datasets/sfcrime_train.csv')

In [ ]:
tomatoes.head()

In [ ]:
crime.head()

## Model Visualization

After you fit a model, you can compute the accuracy of your model by using the `model.score` function.  This will give you back a single number that indicates the quality of the model.  However, this in and of itself may not be terribly useful.

Suppose, you tried a new model and the performance went up.  What phenomena in the data might be driving this change in performance?  Suppose, you actually want to use the machine learning model as a way to gain greater insight into the data.  How can you inspect the model to better understand the relationships between the variables within?

This brings us to the topic of model visualization (also called model introspection).  In the next two sections we will be doing model visualization for two types of models: logistic regression models and decision tree models.

For this section we will use the Rotten Tomatoes dataset.  The specific encoding of text to features that we will be using is [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).  You will recall that we built our own TF-IDF vectorizer in class last week.  Here, we will be using the vectorizer built into scikit learn.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(tomatoes.Phrase)

To better understand what we just did let's take a look at the list of features names for this vectorizer.  These feature names are a list that shows us which column in the TF-IDF output corresponds to which word.  To avoid producing too much output, let's just look at a subset.

In [ ]:
vectorizer.get_feature_names()[1000:1020]

### Visualizing a Logistic Regression Model

Next, we transform the individual phrases using our vectorizer, create a train test split, and fit a logistic regression model.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

X = vectorizer.transform(tomatoes.Phrase)
y = tomatoes.Sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
model = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model.fit(X_train, y_train)
model.score(X_test, y_test)

Next, we'll visualize the learned model using a wordcloud.  First, install the wordcloud generator using pip install.

`pip install wordcloud`

Typically, a word cloud shows words scaled by their frequency in a document.  Here, we will be using the weights learned by our logistic regression model to scale the size of the words.  You can understand these visuals to make words larger if they have a *greater positive contribution to predicting a particular sentiment value*.

Before we do this, let's take a look at our logistic regression model in more detail so we can understand more what's going on under the hood.

In [ ]:
print "coefficients", model.coef_
print "coefficients.shape", model.coef_.shape
print "intercepts", model.intercept_
print "intercepts.shape", model.intercept_.shape

In [ ]:
import math
import numpy as np

print "model probs", model.predict_proba(X_test[0,:])
linear = model.coef_.dot(X_test[0,:].todense().T) + model.intercept_[:,np.newaxis]
print "calculated probs", np.exp(linear) / np.sum(np.exp(linear))

Now that we've built up some intuition for `model.coef_`, let's visualize these coefficients using a wordcloud!

In [ ]:
import wordcloud

def make_word_cloud(feature_names, coefficients):
    """ Create a word cloud with the words given by feature_names
        and given coefficients. """
    cloud = wordcloud.WordCloud()
    cloud.generate_from_frequencies(zip(feature_names, coefficients))
    return cloud.to_image()

In [ ]:
# Word cloud for sentiment value 0 (worst possible)
make_word_cloud(vectorizer.get_feature_names(), model.coef_[0,:])

In [ ]:
# Word cloud for sentiment value 1
make_word_cloud(vectorizer.get_feature_names(), model.coef_[1,:])

In [ ]:
# Word cloud for sentiment value 2
make_word_cloud(vectorizer.get_feature_names(), model.coef_[2,:])

In [ ]:
# Word cloud for sentiment value 3
make_word_cloud(vectorizer.get_feature_names(), model.coef_[3,:])

In [ ]:
# Word cloud for sentiment value 4 (best possible)
make_word_cloud(vectorizer.get_feature_names(), model.coef_[4,:])

### Visualizing a Logistic Regression Model for SF Crime

Next, we'll do a quick spin through the SF Crime dataset to see another example of visualizing a learned model.

In [ ]:
X_crime = pd.get_dummies(crime.DayOfWeek)
y_crime = crime.Category

X_crime_train, X_crime_test, y_crime_train, y_crime_test = \
    train_test_split(X_crime, y_crime, train_size=.05)
model_crime = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model_crime.fit(X_crime_train, y_crime_train)

In [ ]:
%matplotlib inline
import seaborn as sns

sns.plt.figure(figsize=(8,8))
model_crime.classes_
sns.heatmap(model_crime.coef_,
            xticklabels=X_crime.columns,
            yticklabels=model_crime.classes_)
sns.plt.show()

### Visualizing a Decision Tree

Next, we will build a decision tree and use it to visualize the data.  In order to make the visualization tractable, we will make the decision tree relatively small.  However, there are lots of folks that are thinking about how to visualize larger decision trees (even random forests which combine multiple decision trees).  One of these students is our own Michael Bocamazo.  He's on leave doing an LOA, but maybe he'll be willing to provide suggestions down the line.

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(max_depth=4)
tree_model.fit(X_train, y_train)

The easiest way to visualize a decision tree is to simply visualize the individual nodes in the tree graphically.  For small trees this exhaustive approach can work.  We will use the popular library graphviz to do this visualization.  Sklearn has a built in converter from the DecisionTree class to a graphviz dot file.  We will then use some ipython magic to render the graph in our notebook.

In [ ]:
export_graphviz(tree_model, feature_names=vectorizer.get_feature_names(), out_file='tree.dot')

In [ ]:
%install_ext https://raw.github.com/cjdrake/ipython-magic/master/gvmagic.py
%load_ext gvmagic

In [ ]:
f = open('tree.dot')
tree_model_visualization = f.read()
f.close()
%dotstr tree_model_visualization

In [ ]:
tree_model.score(X_test, y_test)

If we look at the first node we are essentially using and as a proxy for a long phrase.  Long phrases are more likely to have polarized sentiment values.  Instead of making the algorithm do the work using TF-IDF, we can instead encode that feature directly.

In [ ]:
from scipy import sparse

num_words_feature = np.asarray(map(lambda x: len(x.split()), tomatoes.Phrase))
num_words_feature = num_words_feature[:, np.newaxis]
X_with_num_words = sparse.hstack((X, num_words_feature))
print X_with_num_words.shape

In [ ]:
tree_model_with_num_words = DecisionTreeClassifier(max_depth=4)
tree_model_with_num_words.fit(X_with_num_words, y)

In [ ]:
export_graphviz(tree_model_with_num_words,
                feature_names=vectorizer.get_feature_names() + ['num_words'],
                out_file='tree.dot')
f = open('tree.dot')
tree_model_visualization = f.read()
f.close()
%dotstr tree_model_visualization

This has been good to visualize what is going on with the decision tree, but let's see how well we can do if we don't limit the depth.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_with_num_words, y, train_size=0.5)
tree_model_with_num_words = DecisionTreeClassifier()
tree_model_with_num_words.fit(X_train, y_train)
tree_model_with_num_words.score(X_test, y_test)

Let's see how good we can do with a Random Forest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_model_with_num_words = RandomForestClassifier()
forest_model_with_num_words.fit(X_train, y_train)
forest_model_with_num_words.score(X_test, y_test)

Now that we've gained some insight that lead us to add a feature consisting of the number of words in the phrase, let's see if that boosts our score using logistic regression.

In [ ]:
model = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model.fit(X_train, y_train)
print "With num words", model.score(X_test, y_test)

model_without_num_words = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model_without_num_words.fit(X_train[:,:-1], y_train)
print "Without num words", model_without_num_words.score(X_test[:,:-1], y_test)

As a final step, let's tune the amount of regularization for our logistic regression model (since it seems to be working the best).

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

model = LogisticRegressionCV(Cs=[math.e**v for v in range(-5,5)],
                             multi_class='multinomial',
                             solver='newton-cg')
model.fit(X_train, y_train)
print "With tuning", model.score(X_test, y_test)

## When to Give Up on a Model?

Why are decision trees generally bad for sentiment analysis?  Let's plot the probability of some sentiment value versus the TF-IDF value for a particular word.  We'll deliberately pick a word that is likely to play an important role in determining the sentiment of a movie review.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def make_log_odds_plot(word, sentiment):
    log_odds = []
    word_index = vectorizer.get_feature_names().index(word)
    word_tf_idf = np.asarray(X_train[:,word_index].todense())
    bucket_centers = np.arange(0, np.max(word_tf_idf), .1)
    bucket_indices = np.digitize(word_tf_idf, bucket_centers)

    for i, v in enumerate(bucket_centers):
        smoothed_prob = np.sum((y_train[:, np.newaxis] == sentiment) * (bucket_indices == i)) / float(sum(bucket_indices == i))
        log_odds.append(np.log(smoothed_prob / (1-smoothed_prob)))
    plt.plot(bucket_centers, log_odds)
    plt.ylabel('log odds ratio sentiment %d' % (sentiment,))
    plt.xlabel('TF-IDF')

make_log_odds_plot('good', 0)
make_log_odds_plot('bad', 0)
plt.legend(['good', 'bad'])
plt.show()

In [ ]:
make_log_odds_plot('good', 4)
make_log_odds_plot('bad', 4)
plt.legend(['good', 'bad'])
plt.show()

From this analysis we see that the there is a strong relationship between the magnitude of a TF-IDF feature and the resultant probability of the phrase having a certain sentiment value.  Why might this be hard for a tree to model?

## Interpreting Log Loss

One complexity of the SF Crime data is that it uses a metric other than accuracy.  This metric is called log loss.  Next, we will take a closer look at that error metric and some issues that arise in interpreting it.

In [ ]:
all_crimes = set(crime.Category)
crime_probabilities = {}
for c in all_crimes:
    crime_probabilities[c] = (crime.Category == c).sum() / float(len(crime.Category))

In [ ]:
crime_probabilities

When evaluating a classifier according to log loss, we require the model to provide a probability of each possible outcome.  The model then incurs a loss proportional to the negative of the log of the probability the model assigned to that outcome.

As an example, suppose the TV weather person assigned a probability of 0.7 to it raining and 0.2 it snowing and 0.1 to neither.  It turned out it snowed.  What is the log loss for this prediction?

### A Sensible Baseline

When we did the titanic dataset, the easiest baseline model was to simply predict that everyone perished (since it was the most common outcome).  Similarly, here we could predict 'OTHER-OFFENSES' for every data point.  However, we must do more than simply predict a single outcome, we must output a probability for each of the possible crimes.  The first thing to try is to output a probability of 1 for 'OTHER-OFFENSES' and 0 for all other crimes.  What will go wrong here?  Note: Kaggle saves you from this without really telling you that it is doing so.

To improve upon this approach, we can simply predict the probabilities observed in the entire dataset for each individual datapoint.

In [ ]:
def baseline(x):
    """ the baseline model ignores x and simply predicts the baseline probabilities
        in the dataset """
    return crime_probabilities

# this is super slow, and not recommended, but I wanted to illustrate the point clearly
ll = 0.0
for row in crime.iterrows():
    # let's just use the X and Y position
    features = np.asarray([row[1].X, row[1].Y])
    probs = baseline(features)
    ll += -np.log(probs[row[1].Category]) / float(len(crime))
    if row[0] % 50000 == 0:
        print row[0]/ float(len(crime))

print "log_loss", ll

This is horribly slow, but luckily we can do a lot better.  You'll notice that we always add the same value for log_loss for a particular category.  To speed things up we can loop over crime categories and multiply the log loss by the count of the number of times it appears in the dataset.

In [ ]:
ll = 0.0
for category in all_crimes:
    # baseline ignores x, so we just pass in None
    probs = baseline(None)
    ll += -np.log(probs[category])*(crime.Category == category).sum() / float(len(crime))

print "log_loss", ll

If we look a little closer, we will see that the second part of the expression in the loop is really just the probability of a crime in the dataset.  This let's us rewrite our code in the following way.

In [ ]:
ll = 0.0
for category, prob in crime_probabilities.items():
    ll += -np.log(prob)*prob

print "log_loss", ll

This may look familiar to you.  This is almost exactly the formula for the [entropy of a discrete probability distribution](https://en.wikipedia.org/wiki/Entropy_%28information_theory%29).  The only difference is that the entropy uses a log with base 2 whereas the log loss typically uses the natural log.  This connection can give you some new ways to think about what this log loss really means.

One interpretation is that the entropy gives us the number of bits of randomness in each event.  Therefore, a crime in San Francisco has $2.68 \times \frac{\ln e}{\ln 2} = 3.87$ bits of randomness associated with it.  Any improvements that you can glean from your model can be quantified by calculating the gain in information that your model provides expressed in bits.  For instance, the leading model on Kaggle right now has a log loss of 2.05 which translates to 2.95 bits of randomness.  The difference betweeen these two is .91 bits.

Other ways to think about Entropy:
1.  On average how surprised are you when an event occurs?
2.  How much data would it take to encode a stream of events?  The inverse being, how much can you compress a stream of data given some underlying knowledge of its structure.